In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv(r"datasets\csv_file.csv")

In [3]:
df=df.head(5000)

In [4]:
ratings = pd.read_csv(r"datasets\ratings_small.csv")

In [5]:
ratings=ratings.head(5000)

In [6]:
len(ratings['movieId'].unique())

2502

In [7]:
links = pd.read_csv(r"datasets\links.csv")

In [8]:
links=links.head(5000)

In [9]:
l_s = pd.read_csv(r"datasets\links_small.csv")

In [10]:
l_s

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


In [11]:
r_s = pd.read_csv(r"datasets\ratings_small.csv")

In [12]:
r_s

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [13]:
l2 = links.merge(ratings,on='movieId')

In [14]:
l2

,movieId,imdbId,tmdbId,userId,rating,timestamp
0,1,114709,862.0,7,3.0,851866703
1,1,114709,862.0,9,4.0,938629179
2,1,114709,862.0,13,5.0,1331380058
3,1,114709,862.0,15,2.0,997938310
4,1,114709,862.0,19,3.0,855190091
...,...,...,...,...,...,...
3573,5071,220627,35650.0,15,5.0,1044348382
3574,5072,293416,9606.0,17,2.5,1127474324
3575,5073,208990,11447.0,23,3.0,1149868375
3576,5074,250081,16550.0,15,1.0,1096005077


In [15]:
l2.rename(columns = {'tmdbId':'id'}, inplace = True)

In [16]:
l2

,movieId,imdbId,id,userId,rating,timestamp
0,1,114709,862.0,7,3.0,851866703
1,1,114709,862.0,9,4.0,938629179
2,1,114709,862.0,13,5.0,1331380058
3,1,114709,862.0,15,2.0,997938310
4,1,114709,862.0,19,3.0,855190091
...,...,...,...,...,...,...
3573,5071,220627,35650.0,15,5.0,1044348382
3574,5072,293416,9606.0,17,2.5,1127474324
3575,5073,208990,11447.0,23,3.0,1149868375
3576,5074,250081,16550.0,15,1.0,1096005077


In [17]:
df = df.merge(l2,on='id')

In [18]:
df.head(3)

,id,title,overview,cast,crew,keywords,genres,movieId,imdbId,userId,rating,timestamp
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",TomHanks TimAllen DonRickles,JohnLasseter,jealousy toy boy friendship friends rivalry bo...,Animation Comedy Family,1,114709,7,3.0,851866703
1,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",TomHanks TimAllen DonRickles,JohnLasseter,jealousy toy boy friendship friends rivalry bo...,Animation Comedy Family,1,114709,9,4.0,938629179
2,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",TomHanks TimAllen DonRickles,JohnLasseter,jealousy toy boy friendship friends rivalry bo...,Animation Comedy Family,1,114709,13,5.0,1331380058


In [19]:
df = df.drop(['imdbId'], axis=1)

In [20]:
df = df[['id','userId','movieId','title','genres','cast','crew','overview','keywords','rating','timestamp']]

In [21]:
df

,id,userId,movieId,title,genres,cast,crew,overview,keywords,rating,timestamp
0,862,7,1,Toy Story,Animation Comedy Family,TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,3.0,851866703
1,862,9,1,Toy Story,Animation Comedy Family,TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,4.0,938629179
2,862,13,1,Toy Story,Animation Comedy Family,TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,5.0,1331380058
3,862,15,1,Toy Story,Animation Comedy Family,TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,2.0,997938310
4,862,19,1,Toy Story,Animation Comedy Family,TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,3.0,855190091
...,...,...,...,...,...,...,...,...,...,...,...
3568,35650,15,5071,Maelström,Drama,Marie-JoséeCroze Jean-NicolasVerreault Stephan...,DenisVilleneuve,Bibiane is the daughter of a well-known fashio...,independentfilm,5.0,1044348382
3569,9606,17,5072,Metropolis,Animation ScienceFiction,YukaImoto KeiKobayashi KoukiOkada,Rintaro,Duke Red has overseen the construction of a ma...,jealousy metropolis future son control basedon...,2.5,1127474324
3570,11447,23,5073,The Son's Room,Drama,NanniMoretti LauraMorante JasmineTrinca,NanniMoretti,A psychoanalyst and his family go through prof...,profession sadness therapist family'sdailylife...,3.0,1149868375
3571,16550,15,5074,Storytelling,Comedy Drama,JohnGoodman SelmaBlair RobertWisdom,ToddSolondz,College and high school serve as the backdrop ...,sex exploitation highschool celebrity independ...,1.0,1096005077


In [22]:
df.isnull().sum()

id            0
userId        0
movieId       0
title         0
genres        1
cast          1
crew          0
overview      0
keywords     31
rating        0
timestamp     0
dtype: int64

In [23]:
df.dropna(inplace=True)

In [24]:
df.duplicated().sum()

0

In [25]:
df.drop_duplicates(keep=False, inplace=True)

In [26]:
df

,id,userId,movieId,title,genres,cast,crew,overview,keywords,rating,timestamp
0,862,7,1,Toy Story,Animation Comedy Family,TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,3.0,851866703
1,862,9,1,Toy Story,Animation Comedy Family,TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,4.0,938629179
2,862,13,1,Toy Story,Animation Comedy Family,TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,5.0,1331380058
3,862,15,1,Toy Story,Animation Comedy Family,TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,2.0,997938310
4,862,19,1,Toy Story,Animation Comedy Family,TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ...",jealousy toy boy friendship friends rivalry bo...,3.0,855190091
...,...,...,...,...,...,...,...,...,...,...,...
3568,35650,15,5071,Maelström,Drama,Marie-JoséeCroze Jean-NicolasVerreault Stephan...,DenisVilleneuve,Bibiane is the daughter of a well-known fashio...,independentfilm,5.0,1044348382
3569,9606,17,5072,Metropolis,Animation ScienceFiction,YukaImoto KeiKobayashi KoukiOkada,Rintaro,Duke Red has overseen the construction of a ma...,jealousy metropolis future son control basedon...,2.5,1127474324
3570,11447,23,5073,The Son's Room,Drama,NanniMoretti LauraMorante JasmineTrinca,NanniMoretti,A psychoanalyst and his family go through prof...,profession sadness therapist family'sdailylife...,3.0,1149868375
3571,16550,15,5074,Storytelling,Comedy Drama,JohnGoodman SelmaBlair RobertWisdom,ToddSolondz,College and high school serve as the backdrop ...,sex exploitation highschool celebrity independ...,1.0,1096005077


In [27]:
# Jaccard similarity
# Cosine similarity
# Pearson Correlation(Centered cosine) - advantage of clear seperation b/w tough raters and easy raters
# In practice, item-item outperforms user-user filtering mostly because items are of small group of similarities(genres)
# but users have varied tastes, hence item similarity has more sense than user similarity


In [28]:
df = df[['userId','movieId','title','rating']]

In [29]:
df

,userId,movieId,title,rating
0,7,1,Toy Story,3.0
1,9,1,Toy Story,4.0
2,13,1,Toy Story,5.0
3,15,1,Toy Story,2.0
4,19,1,Toy Story,3.0
...,...,...,...,...
3568,15,5071,Maelström,5.0
3569,17,5072,Metropolis,2.5
3570,23,5073,The Son's Room,3.0
3571,15,5074,Storytelling,1.0


In [30]:
table = df.pivot_table(index = ["userId"],columns = ["title"],values = "rating")
table

title,10 Things I Hate About You,12 Angry Men,15 Minutes,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,28 Days,42 Up,8 Heads in a Duffel Bag,8 ½ Women,...,Yellow Submarine,Yojimbo,You Can Count on Me,You've Got Mail,Young Frankenstein,Young Guns,Zero Effect,Zoolander,eXistenZ,¡Three Amigos!
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
table = table.fillna(0)

In [32]:
table

title,10 Things I Hate About You,12 Angry Men,15 Minutes,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,28 Days,42 Up,8 Heads in a Duffel Bag,8 ½ Women,...,Yellow Submarine,Yojimbo,You Can Count on Me,You've Got Mail,Young Frankenstein,Young Guns,Zero Effect,Zoolander,eXistenZ,¡Three Amigos!
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
def standardize(row):
    new_row = (row - row.mean())/(row.max() - row.min())
    return new_row

new_table = table.apply(standardize)

In [34]:
new_table

title,10 Things I Hate About You,12 Angry Men,15 Minutes,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,28 Days,42 Up,8 Heads in a Duffel Bag,8 ½ Women,...,Yellow Submarine,Yojimbo,You Can Count on Me,You've Got Mail,Young Frankenstein,Young Guns,Zero Effect,Zoolander,eXistenZ,¡Three Amigos!
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.060714,-0.153571,-0.047619,-0.089286,-0.207143,-0.035714,-0.035714,-0.035714,-0.035714,-0.035714,...,-0.035714,-0.035714,-0.067857,-0.075893,-0.085714,-0.080357,-0.035714,-0.071429,-0.085714,-0.035714
2,-0.060714,-0.153571,-0.047619,-0.089286,-0.207143,-0.035714,-0.035714,-0.035714,-0.035714,-0.035714,...,-0.035714,-0.035714,-0.067857,-0.075893,-0.085714,-0.080357,-0.035714,-0.071429,-0.085714,-0.035714
3,-0.060714,-0.153571,-0.047619,-0.089286,-0.207143,-0.035714,-0.035714,-0.035714,-0.035714,-0.035714,...,-0.035714,-0.035714,-0.067857,-0.075893,-0.085714,0.919643,-0.035714,-0.071429,-0.085714,-0.035714
4,-0.060714,-0.153571,-0.047619,-0.089286,-0.207143,-0.035714,-0.035714,-0.035714,-0.035714,-0.035714,...,-0.035714,-0.035714,-0.067857,-0.075893,0.914286,-0.080357,-0.035714,-0.071429,-0.085714,-0.035714
5,-0.060714,-0.153571,-0.047619,-0.089286,-0.207143,-0.035714,-0.035714,-0.035714,-0.035714,-0.035714,...,-0.035714,-0.035714,-0.067857,0.924107,-0.085714,-0.080357,-0.035714,-0.071429,-0.085714,-0.035714
6,-0.060714,-0.153571,-0.047619,-0.089286,-0.207143,-0.035714,-0.035714,-0.035714,-0.035714,-0.035714,...,-0.035714,-0.035714,-0.067857,-0.075893,-0.085714,-0.080357,-0.035714,-0.071429,-0.085714,-0.035714
7,-0.060714,-0.153571,-0.047619,-0.089286,0.592857,-0.035714,-0.035714,-0.035714,-0.035714,-0.035714,...,-0.035714,-0.035714,-0.067857,-0.075893,0.514286,-0.080357,-0.035714,-0.071429,-0.085714,-0.035714
8,-0.060714,-0.153571,-0.047619,-0.089286,-0.207143,-0.035714,-0.035714,-0.035714,-0.035714,-0.035714,...,-0.035714,-0.035714,-0.067857,-0.075893,-0.085714,-0.080357,-0.035714,-0.071429,-0.085714,-0.035714
9,-0.060714,-0.153571,-0.047619,-0.089286,-0.207143,-0.035714,-0.035714,-0.035714,-0.035714,-0.035714,...,0.964286,-0.035714,-0.067857,-0.075893,-0.085714,-0.080357,-0.035714,-0.071429,-0.085714,-0.035714


In [35]:
item_similarity = cosine_similarity(new_table.T)

In [36]:
item_similarity

array([[1.        , 0.26120567, 0.20938724, ..., 0.54745666, 0.28799079,
        0.81225084],
       [0.26120567, 1.        , 0.53639714, ..., 0.5833645 , 0.35427069,
        0.37572817],
       [0.20938724, 0.53639714, 1.        , ..., 0.88762536, 0.64602029,
        0.28426762],
       ...,
       [0.54745666, 0.5833645 , 0.88762536, ..., 1.        , 0.67302618,
        0.69388867],
       [0.28799079, 0.35427069, 0.64602029, ..., 0.67302618, 1.        ,
        0.39098113],
       [0.81225084, 0.37572817, 0.28426762, ..., 0.69388867, 0.39098113,
        1.        ]])

In [37]:
new_df = pd.DataFrame(item_similarity,index=new_table.columns,columns=new_table.columns)

In [38]:
new_df

title,10 Things I Hate About You,12 Angry Men,15 Minutes,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,28 Days,42 Up,8 Heads in a Duffel Bag,8 ½ Women,...,Yellow Submarine,Yojimbo,You Can Count on Me,You've Got Mail,Young Frankenstein,Young Guns,Zero Effect,Zoolander,eXistenZ,¡Three Amigos!
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,1.000000,0.261206,0.209387,-0.093464,0.287620,0.812251,0.812251,0.812251,0.812251,0.812251,...,-0.052503,0.812251,0.579389,0.168220,-0.092374,-0.090813,0.812251,0.547457,0.287991,0.812251
12 Angry Men,0.261206,1.000000,0.536397,0.420532,0.781598,0.375728,0.375728,0.375728,0.375728,0.375728,...,-0.089261,0.375728,0.580295,0.298162,0.115655,0.024336,0.375728,0.583364,0.354271,0.375728
15 Minutes,0.209387,0.536397,1.000000,-0.084341,0.386741,0.284268,0.284268,0.284268,0.284268,0.284268,...,-0.047378,0.284268,0.861186,0.608851,-0.083357,-0.081949,0.284268,0.887625,0.646020,0.284268
"20,000 Leagues Under the Sea",-0.093464,0.420532,-0.084341,1.000000,0.334428,-0.065932,-0.065932,-0.065932,-0.065932,-0.065932,...,-0.065932,-0.065932,-0.094877,-0.110800,0.317073,0.489113,-0.065932,-0.095019,-0.116002,-0.065932
2001: A Space Odyssey,0.287620,0.781598,0.386741,0.334428,1.000000,0.422120,0.422120,0.422120,0.422120,0.422120,...,-0.110284,0.422120,0.498563,0.193603,0.243101,-0.027042,0.422120,0.493249,0.609983,0.422120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Young Guns,-0.090813,0.024336,-0.081949,0.489113,-0.027042,-0.064062,-0.064062,-0.064062,-0.064062,-0.064062,...,-0.064062,-0.064062,-0.092186,-0.107657,0.121061,1.000000,-0.064062,-0.092323,-0.112712,-0.064062
Zero Effect,0.812251,0.375728,0.284268,-0.065932,0.422120,1.000000,1.000000,1.000000,1.000000,1.000000,...,-0.037037,1.000000,0.732127,0.245303,-0.065164,-0.064062,1.000000,0.693889,0.390981,1.000000
Zoolander,0.547457,0.583364,0.887625,-0.095019,0.493249,0.693889,0.693889,0.693889,0.693889,0.693889,...,-0.053376,0.693889,0.998512,0.575128,-0.093911,-0.092323,0.693889,1.000000,0.673026,0.693889


In [39]:
def recommend_movie(movie_name,user_rating):
    score = new_df[movie_name]*(user_rating-2.5)
    score = score.sort_values(ascending = False).head(15)
    return score

In [40]:
while True:
    print(recommend_movie(input(),int(input())))

title
Toy Story                   2.500000
The Shawshank Redemption    1.505396
Boys Don't Cry              1.310213
From Hell                   1.262079
Pearl Harbor                1.243935
Fargo                       1.235649
Shadowlands                 1.189945
The Matrix                  1.187449
Half Baked                  1.175085
True Lies                   1.172078
The Secret Garden           1.168205
The Boondock Saints         1.162959
A Close Shave               1.154407
Sliding Doors               1.137002
To Kill a Mockingbird       1.132330
Name: Toy Story, dtype: float64


ValueError: invalid literal for int() with base 10: ''

In [41]:
import pickle

In [42]:
pickle.dump(new_df.to_dict(),open('movies_collaborative_dict.pkl','wb'))

In [43]:
pickle.dump(item_similarity,open('similarity_collaborative.pkl','wb'))

In [44]:
import bz2

In [ ]:
def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f: 
        pickle.dump(data, f)
compressed_pickle('movies_collaborative_dict', new_df) 